# Spring 2019 | CS 6400

Author: Travis Jefferies
Last Updated: 04252019

## Efficiency

Your database could be the best designed thing in the world, but if it isn't fast enough, nobody will use it! We must first begin with some computer hardware 101.

### Computer Architecture 101

![](p1.svg)

Main memory - RAM - volatile, fast, expensive \$\$\$<br>
Secondary Memory - DISK - permanent, slow, big and cheap $<br>
* Applications run by the CPU can only query and update data in main memory
* Data must be written back to secondary memory after being updated
* Only tiny fraction of a real database fits into main memory

### Who Cares?

We should care about this stuff when considering the difference in access time between main memory and disk:
* Main Memory
    * 30ns or $3\times10^{-7}$ sec
* Disk
    * 10ms or $1\times10^{-2}$ sec

Assuming we have 60 seconds to access things, its $\frac{\frac{60}{3\times10^{-7}}}{\frac{60}{0.01}}$ or 33,333$\times$ faster to use main memory.
* In fact, RAM access time is completely ignored when computing computational cost
    * Only disk access time is even considered!
    * CPU cost is ignored too in these calculations
    
### Disk

See picture below for an in-depth overview of the disk:

![](p2.png)

A disk has a number of plates or platters.<br>
For each one of the platters, there is a read-write head that accesses the top of the plate, and one that accesses the bottom of the plate.<br>
All of the read-write heads are connected together and are operated by an actuator.<br>
On each surface, what passes under the read-write in that position is called a track.<br>
A collection of tracks is called a cylinder.<br>
Each surface is split up into a number of sectors.<br>
A sector is the smallest physical unit that could be transported from disk to main memory.
* Usually consists  of 512 bytes

Blocks have contributions from several sectors.<br>
* Typically blocks are 4k bytes or 8 sectors
    * Could be 8k or 16k depending on the data we are storing
    
### Records, Blocks, Files

The memory access required for database applications can mainly be thought about as records, blocks, and files.

#### Records

Records are stored on a block on a disk.<br>

![](p3.png)

RegularUser(
    <u>Email</u> varchar(50),
    Sex char(1),
    BirthDate datetime,
    CurrentCity varchar(50),
    Hometown varchar(50)
)

datetime = 8 bytes<br>
record size = 50 + 1 + 8 + 50 + 50 = [1, 160] = 159 bytes<br>

#### Blocks

Now if we assume the following specs:

block size: 4 kb (+metadata)<br>
filled: ~80%<br>

We can expect the following memory footprint:

4,000 $\times$ 0.8 = 3,200<br>
3,200 / 159 is = 20.126 records/block<br>

So what do we do with the remaining 0.874 record?

![](p4.png)

If we decide to go with the option on the left - where 0.126 of one record starts in one block and the other 0.874 record is represented in another block, we have what is called a *spanned representation*.<br>

The default action in most database systems is to run with unspanned representations, simply to avoid the processing that's needed to break off records.
* Obviously if you have record sizes that are larger than block sizes than you don't have a chice but to run with the spanned representation.

Now that we have the concept of Blocks, we can create Files.

#### Files

Files are Blocks linked together by pointers.<br>

![](p5.png)

Assuming the following specs:

block pointer size: 4 bytes (true for 32 bit architecture)<br>
\# Records: 4,000,000<br>
block size: 4 kb

Assuming we can fit ~20 records/block, 4,000,000 records will require $\frac{4,000,000 \text{records}}{20 \frac{\text{records}}{block}} \approx 200,000$ Blocks

So we can expect ~$200,000 \times 4\text{kb} \approx 800\text{MB}$ size file on disk.

### Compute Transport Time from Disk to Main Memory

Assumptions:

* Seek time: 3-8 ms
* Rotational Delay: 2-3 ms
* Transfer time: 0.5-1.0 ms
* Total: 5-12 ms

0.01 sec or 10 ms per page fault.<br>

What if instead of picking up 1 block at a time, we chose to pick up 250 blocks per seek-rotation?<br>
These are called ***extent transfers***.

Normally such an operation would cost $250 \text{ blocks} \times 10\frac{\text{ ms}}{\text{block}} \approx 2.5 \text{ secs}$.<br>

Using *extent transfers* we can do the same operation for $\approx 0.25 \text{ secs}$.
* Extent transfers only incur seek time and rotational delay on finding the first block
    * From that point on it is straight transfer time, seek-rotation delay is eliminated
    
The downside to using *extent transfers* is we will probably need more buffer space.<br>
Whenever we move data from the disk into main memory or from main memory back to disk, we need good buffer management strategies.
* One of the most commonly used strategies is Least Recently Used (LRU)
    * If we run out of buffer space and need to free up space in main memory for the data being transfered from disk, we find what is least recently used and overwrite that.
    * Philosophy being *We haven't used it in awhile, were probably not gonna use it next."
    
LRU is ideal for merge joins.

* LRU really struggles with nested loop joins
    * MRU or Most Recently Used is ideal for nested loop joins
    
### Sorting Data

A main use case for a computerized database is the ability to sort data. It's in our best interest to do this optimally whenever possible.

#### Heap - Unsorted File

A heap is an unsorted file. Records sit on a bunch of blocks and there is no sorting amongst the records.
* Any record we are looking for in the whole file could sit on any block whatsoever
* Lookup Time: $N/2$ for linear seach where $N = $ # of data blocks

Assumptions:
* block pointer: 4 bytes
* \# records: 4,000,000
* \# data blocks: ~200,000
* file size: ~800MB


Using the assumptions from above,
$\frac{200,000}{2} \times 0.01s = 16.6$ minutes

In the real world of search, this would be like crumpling up the pages of a phonebook, throwing the crumpled pages into a bag, and pulling out page-by-crumpled-page until you found the name you were searching for. ***We can do better!***
* A heap can be useful for queuing situations where you are only poping and appending

#### Sorted File + Binary Search

Binary search is super magical. You'll use it alot. ***Data must first be sorted before using binary search!***

In the real world of search, binary search is like looking for a name in a phonebook by taking the phonebook and splitting (depending on if the middle value is higher or lower than your query) it in half again and again until you find the name you are searching for.
* Lookup Time: $\log_{2}\left(N\right)$ for 2-ary search where $N = $ # of data blocks

In a database context, our ***data must first be sorted*** and will thus have the following block layout:

![](p6.png)

Binary search will first search the middle block of memory, then will split (depending on if the middle value is higher or lower than your query) in half again and again until it finds the record you are searching for.

Assumptions:
* block pointer: 4 bytes
* \# records: 4,000,000
* \# data blocks: ~200,000
* file size: ~800MB

Using the assumptions from above,
$\log_{2}\left(200000\right) \times 0.01s = 0.18$ seconds

This means that for every single search we do using linear search, we could be doing $\frac{16.6 \text{ minutes } \times \, 60 \frac{\text{ seconds per}}{\text{minute}}}{0.18 \text{ seconds}} = 5533 \times$ as many binary searches.

!!

### Primary & Secondary Index

#### Primary Indices

Besides using smarter sorting and searching algorithms, we may want to consider ways of reducing the size of what we're going to be searching in. ***Enter the indices!*** First we'll talk about primary indices, then we'll talk about secondary indices.

Primary indices are great for looking up a particular record with a particular value.<br>
They are also great for doing range queries - the use of pointers in the structure allows us to find the first value in a range, then follow the values at the data level or at the index level.<br>
* Lookup Time: $\log_{2}\left(n\right) + 1$ where $n = $ \# index blocks

![](p8.svg)

To build a primary index, we march down the sorted collection of memory blocks, reading the key from record 1, record 2, record 3, $\ldots$ , record n.<br>
Each time we read a key, we make a copy of the key and insert the copy into the index (k1 above).<br>
Additionally, we store a pointer back to the original memory block the key was read from (blank cell with pointer next to k1 above).<br>
When that index block is full, we continue to the next index block.<br>
This continues until all keys from all records have been read and inserted into the index.
* Notice that we are reading keys from sorted records... this means that the index is born sorted - no need to sort the primary index after creation!

Also notice how we are only copying a single key over, NOT the entire relation. This allows us to pack alot of records into an index block. How many records can we stuff into a single index block?

Given:

RegularUser(
    <u>Email</u> varchar(50),
    Sex char(1),
    BirthDate datetime,
    CurrentCity varchar(50),
    Hometown varchar(50)
)

Assumptions:
* block pointer: 4 bytes
* block memory footprint: 4 kb
* filled: ~80%
* \# data blocks: ~200,000
* \# records: 4,000,000
* file size: ~800MB

So we have $4000 \times 0.8 = 3200$  bytes allocated to each index block.<br>

Our PK field RegularUser.Email takes up 50 bytes.<br>
Let's add in 4 bytes for each memory block pointer.<br>
$50+4=54$ bytes per key.

So $\frac{\frac{3200 \text{ bytes per}}{\text{block}}}{54 \frac{\text{ bytes per}}{\text{key}}} = 59.26$ keys per block. This value is referred to as the ***fanout*** of the primary index. For practical purposes, the fanout in this case is ~60, with an 80% block memory space utilization.
* It is called the "fanout" because thats how many pointers will "fan" "out" from the index memory block to the source memory blocks

Ok, so how many index blocks do we need?

Well we need to point to ~200,000 blocks.<br>
We can point to 60 blocks from a single index block.<br>
$\frac{200,000 \text{ blocks}}{60 \frac{\text{}}{\text{}}} \approx 3334$ index blocks in the sparse case.

The lookup time for the sparse case is:<br>
$\left(\log_{2}\left(200,000/60\right) + 1\right) * 0.01\text{ seconds} = \left(12+1\right) * 0.01\text{ seconds} = 0.13 \text{ seconds}$.
* Notice that this 0.13 seconds is actually cheaper than the binary search case (0.18 seconds)

Sparse case? Does that mean there is a dense case? Yup.
* \# index blocks (sparse): 3334

For the dense case, we simply read and copy each and every key found in the 4,000,000 records to the index blocks.<br>
This index has a much larger memory footprint (20$\times$ the sparse case) on disc.<br>
* \# index blocks (dense): 66,666

This is advantageous in the sense that some queries can be asked that do not need to access data, but instead can just access the index.
* For example, what's the maximum key value, the minimum key value, average key value, etc.
    * This is determined by a query on the index alone!

The lookup time for the dense case is:<br>
$\left(\log_{2}\left(4,000,000/60\right) + 1\right) * 0.01\text{ seconds} = \left(16+1\right) * 0.01\text{ seconds} = 0.17 \text{ seconds}$.
* Notice that this 0.17 seconds is still cheaper than the binary search case (0.18 seconds)

Everything that we have talked about thus far is still only *lookup cost*.<br>
Update cost is typically twice as much cost, but may vary depending on the underlying data structure of the index.

#### Secondary Indices



In [7]:
import math
(16.6*60)/0.18
#log(200000)

5533.333333333334